In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
data = pd.read_csv('./data/train_dataset.csv', index_col=0)
data.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
id,,,,,,,,,,,,,,,,,,,,,
1,Male,disloyal Customer,33,Business travel,Eco,571,2,3,2,4,...,4,3,1,3,4,3,4,10,3.0,0
2,Female,Loyal Customer,49,Business travel,Business,1431,4,1,4,4,...,5,5,5,5,3,5,3,0,0.0,1
3,Female,Loyal Customer,43,Business travel,Eco,867,1,4,4,4,...,1,1,1,1,1,1,2,0,18.0,0
4,Female,Loyal Customer,27,Business travel,Business,1550,3,3,3,3,...,2,4,4,5,5,4,2,0,0.0,1
5,Male,Loyal Customer,11,Personal Travel,Eco,526,3,4,3,2,...,4,5,2,5,3,5,4,0,10.0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1 to 10000
Data columns (total 23 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Gender                             10000 non-null  object 
 1   Customer Type                      10000 non-null  object 
 2   Age                                10000 non-null  int64  
 3   Type of Travel                     10000 non-null  object 
 4   Class                              10000 non-null  object 
 5   Flight Distance                    10000 non-null  int64  
 6   Inflight wifi service              10000 non-null  int64  
 7   Departure/Arrival time convenient  10000 non-null  int64  
 8   Ease of Online booking             10000 non-null  int64  
 9   Gate location                      10000 non-null  int64  
 10  Food and drink                     10000 non-null  int64  
 11  Online boarding                    10000 non-null  int

In [4]:
data.notnull().sum()

Gender                               10000
Customer Type                        10000
Age                                  10000
Type of Travel                       10000
Class                                10000
Flight Distance                      10000
Inflight wifi service                10000
Departure/Arrival time convenient    10000
Ease of Online booking               10000
Gate location                        10000
Food and drink                       10000
Online boarding                      10000
Seat comfort                         10000
Inflight entertainment               10000
On-board service                     10000
Leg room service                     10000
Baggage handling                     10000
Checkin service                      10000
Inflight service                     10000
Cleanliness                          10000
Departure Delay in Minutes           10000
Arrival Delay in Minutes              9972
satisfaction                         10000
dtype: int6

In [5]:
cat_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class',]
num_cols = [
    'Flight Distance', 'Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location',
    'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service',
    'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

num_pipline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler', StandardScaler()),
])

pipeline = ColumnTransformer([
    ('nums', num_pipline, num_cols),
    ('cats', OneHotEncoder(), cat_cols)
])

In [7]:
X_train = data.drop('satisfaction', axis=1)
X_train = pipeline.fit_transform(X_train)
y_train = data['satisfaction'].copy()

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Std.dev:", scores.std())


In [11]:
DT_model = DecisionTreeClassifier()
DT_scores = cross_val_score(DT_model, X_train, y_train, cv=15)
display_scores(DT_scores)

Scores: [0.94152924 0.92953523 0.93553223 0.93103448 0.91904048 0.93103448
 0.95052474 0.93553223 0.92353823 0.90404798 0.91891892 0.93993994
 0.91441441 0.93543544 0.93393393]
Mean: 0.9295994645319983
Std.dev: 0.011431667608280079


In [13]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

param_grid = [
        {
            'activation' : ['identity', 'logistic', 'tanh', 'relu'],
            'solver' : ['lbfgs', 'sgd', 'adam'],
            'hidden_layer_sizes': np.arange(1, 21)
        }
       ]

clf = GridSearchCV(MLPClassifier(), param_grid, cv=3,
                           scoring='accuracy')
clf.fit(X_train,y_train)

/home/doni/miniconda3/envs/ml/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/doni/miniconda3/envs/ml/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/doni/miniconda3/envs/ml/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/doni/miniconda3/envs/ml/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/doni/m

GridSearchCV(cv=3, estimator=MLPClassifier(),
             param_grid=[{'activation': ['identity', 'logistic', 'tanh',
                                         'relu'],
                          'hidden_layer_sizes': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20]),
                          'solver': ['lbfgs', 'sgd', 'adam']}],
             scoring='accuracy')

In [14]:
clf.best_score_

0.9388995588221091

In [9]:
from sklearn.ensemble import RandomForestClassifier

RF_model = RandomForestClassifier()
RF_scores = cross_val_score(RF_model, X_train, y_train, cv=15)
display_scores(RF_scores)

Scores: [0.95352324 0.95352324 0.95352324 0.94452774 0.95502249 0.94002999
 0.96101949 0.95502249 0.94752624 0.94902549 0.95045045 0.95795796
 0.93393393 0.95645646 0.95495495]
Mean: 0.9510998254626439
Std.dev: 0.0069054532805688535


In [16]:
from sklearn.neighbors import KNeighborsClassifier

KN_model = KNeighborsClassifier(n_neighbors=5)
KN_scores = cross_val_score(KN_model, X_train, y_train, cv=15)
display_scores(KN_scores)

Scores: [0.90404798 0.89805097 0.91004498 0.91304348 0.90254873 0.91304348
 0.91454273 0.92053973 0.89205397 0.91154423 0.92492492 0.91441441
 0.89339339 0.91891892 0.92192192]
Mean: 0.9102022562292428
Std.dev: 0.009823465770379992


In [17]:
from sklearn.ensemble import GradientBoostingClassifier

GB_model = KNeighborsClassifier(n_neighbors=5)
GB_scores = cross_val_score(GB_model, X_train, y_train, cv=15)
display_scores(GB_scores)

Scores: [0.90404798 0.89805097 0.91004498 0.91304348 0.90254873 0.91304348
 0.91454273 0.92053973 0.89205397 0.91154423 0.92492492 0.91441441
 0.89339339 0.91891892 0.92192192]
Mean: 0.9102022562292428
Std.dev: 0.009823465770379992


### RandomForest modeli eng maqul deb topildi

In [20]:
RF_model.fit(X_train, y_train)

RandomForestClassifier()

In [12]:
test_data = pd.read_csv('./data/test_dataset.csv', index_col=0)
sample = pd.read_csv('./data/sample_submission.csv', index_col=0)

In [15]:
test_data.isnull().sum()

Gender                                0
Customer Type                         0
Age                                   0
Type of Travel                        0
Class                                 0
Flight Distance                       0
Inflight wifi service                 0
Departure/Arrival time convenient     0
Ease of Online booking                0
Gate location                         0
Food and drink                        0
Online boarding                       0
Seat comfort                          0
Inflight entertainment                0
On-board service                      0
Leg room service                      0
Baggage handling                      0
Checkin service                       0
Inflight service                      0
Cleanliness                           0
Departure Delay in Minutes            0
Arrival Delay in Minutes             19
dtype: int64

In [16]:
X = pipeline.fit_transform(test_data)

In [21]:
sample['satisfaction'] = RF_model.predict(X)

In [ ]:
sample.to_csv('my_solution.csv')